In [1]:
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")

c:\Users\Paolo\Documents\carol_emo_rec\MLLM\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [02:22<00:00, 28.56s/it]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [2]:
from datasets import load_dataset

from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
import librosa
from datasets import load_dataset
import torch

# Replace 'your_dataset_name' with the name of your Hugging Face dataset
dataset = load_dataset("cairocode/IEMO_WAV_002")


import torch
import numpy as np
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
import librosa
import soundfile as sf
from io import BytesIO

# Define the emotion mapping
emotion_mapping = {
    "Neutral": 0,
    "Happy": 1,
    "Sad": 2,
    "Angry": 3,
}

In [3]:
from datasets import concatenate_datasets

# Define the filter function with an argument
def create_filter_function(spk_id):
    def filter_m_examples(example):
        return example["label"] != 4 and example["label"] != 5 and example["speakerID"] == spk_id
    return filter_m_examples

# Use the filter function with the specific speaker ID
spk_id = 2
filter_function = create_filter_function(spk_id)
ds_002 = dataset['train'].filter(filter_function)

In [4]:
ds = ds_002

In [7]:
from datasets import Dataset
from collections import defaultdict
import random

def group_by_label(dataset, label_key):
    """Group dataset samples by label."""
    label_groups = defaultdict(list)
    for sample in dataset:
        label = sample[label_key]
        label_groups[label].append(sample)
    return label_groups

def create_balanced_batches(dataset, label_key, batch_size):
    """Create balanced batches ensuring at least one sample per label."""
    label_groups = group_by_label(dataset, label_key)
    batches = []
    labels = list(label_groups.keys())
    
    while any(len(label_groups[label]) > 0 for label in labels):
        batch = []
        random.shuffle(labels)  # Shuffle labels for randomness
        for label in labels:
            if len(batch) < batch_size and len(label_groups[label]) > 0:
                batch.append(label_groups[label].pop(0))  # Take one sample
        
        # Fill the remaining spots in the batch
        remaining_samples = [
            sample for label in labels
            for sample in label_groups[label][:max(0, batch_size - len(batch))]
        ]
        random.shuffle(remaining_samples)  # Add randomness to the selection
        batch.extend(remaining_samples[:batch_size - len(batch)])
        
        batches.append(batch)
    
    return batches

def balanced_batches_as_datasets(dataset, label_key, batch_size):
    """Create balanced batches and return them as a list of Dataset objects."""
    batches = create_balanced_batches(dataset, label_key, batch_size)
    dataset_batches = [
        Dataset.from_dict({key: [sample[key] for sample in batch] for key in dataset.column_names})
        for batch in batches
    ]
    return dataset_batches

# Example usage
# Assuming `train_dataset` is a Hugging Face Dataset and labels are in the 'label' column
batch_size = 8
balanced_dataset_batches = balanced_batches_as_datasets(ds, 'label', batch_size)

# Combine batches back into a single Dataset if needed
final_dataset = Dataset.from_dict({
    key: [sample for batch in balanced_dataset_batches for sample in batch[key]]
    for key in ds.column_names
})

print(final_dataset)


Dataset({
    features: ['audio', 'label', 'valence', 'arousal', 'domination', 'arousal_norm', 'valence_norm', 'speakerID', 'utterance_id', 'transcript', 'speaker_id'],
    num_rows: 1340
})


In [11]:
1


0 8 range(0, 8)
 | Actual Label: 3 | Response: Understood, I will use the provided information to better understand the emotions expressed by the person. | Transcript: That's out of control.
 | Actual Label: 1 | Response: Happy | Transcript: Well Vegas was awesome.
 | Actual Label: 2 | Response: Based on the tone and context provided, it's likely that the speaker is feeling sad. This can be inferred from the fact that they mentioned getting the mail and asking if she saw their letter, which suggests a sense of longing or disappointment. Additionally, the use of a sigh at the end of the sentence further emphasizes a feeling of sadness. | Transcript: Did you get the mail? So you saw my letter?
 | Actual Label: 0 | Response: Understood, I will use the provided information to better understand the person's emotions in future interactions. | Transcript: Excuse me.
 | Actual Label: 0 | Response: Neutral | Transcript: Clearly.  You know, do you have like a supervisor or something?
 | Actual L

IndexError: list index out of range